lizon's posts topic modeling using BERTopic, topics entitlement and summarization. Not very good results. Need to try Top2Vec instead and more efforts summarization is requred.

In [ ]:
!pip install bertopic

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from bertopic import BERTopic

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
Data = '/content/drive/MyDrive/Colab Notebooks/Projects/eva/Data/'

Messages_filename='all_lizon_and_clones_messages.csv'
Messages_full_filename=os.path.join(Data, Messages_filename)



text='message'



Experiments_file='/content/drive/MyDrive/Colab Notebooks/Projects/eva/ExperimentLogs/lizon_topics.xlsx'

In [26]:
#Saving into log (Excel file)
import openpyxl 
def SaveToExperimentLog(Experiments_file, LogEntry, data):
    book = openpyxl.load_workbook(Experiments_file)
    writer = pd.ExcelWriter(Experiments_file, engine='openpyxl') 
    writer.book = book

    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

    data.to_excel(writer, LogEntry[0:29],index=False)

    writer.save()
    writer.close()

In [4]:
df=pd.read_csv(Messages_full_filename, error_bad_lines=False, index_col=False)

In [ ]:
#only relatively long messages (25+ words)
df=df[df['message_words']>=25]

In [ ]:
#change text column type to string
df[text]=df[text].values.astype(str)

In [ ]:
import nltk

In [ ]:

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
ru_stopwords = stopwords.words('russian')
eng_stopwords = stopwords.words('english')
add_stopwords = ['ай',
'ну',
'по',
'это',
'так',
'что',
'вот',
'кого',
'то',
'кто',
'все',
'ваш','жизнь','жить','свой',
'сообщение','распечатать','manu','сообщения'
	]
stopwords = eng_stopwords + ru_stopwords + add_stopwords

from collections import Counter
stopwords_dict = Counter(stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 3.8 MB/s  eta 0:00:01
     |████████████████████████████████| 8.2 MB 46.2 MB/s 


In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return  ' '.join(res)
def text_preprocess(row):
  s=lemmatize(row[text])
  return ' '.join([word for word in tokenizer.tokenize(s) if word not in stopwords_dict])

In [ ]:
df['preprocessed_message']=df.apply(text_preprocess,axis=1)

In [ ]:
clones_topics_df=pd.DataFrame()
for Author_Id,author in zip(df['Author_Id'].unique(),df['author'].unique()):
  data = df[df['Author_Id']==Author_Id]['preprocessed_message'].tolist()
  #model = BERTopic(embedding_model='blinoff/roberta-base-russian-v0')
  model = BERTopic(language="Russian",nr_topics=10)
  topics, probabilities = model.fit_transform(data)
  #add to combine dataset
  topics_df=model.get_topic_info()
  #num_words=topics_df['Name'].values[0].count('_')
  num_words=10
  for i in range(0,num_words):
    topics_df['w'+str(i)]=''
  for i in range(0,num_words):  
    topics_df['p'+str(i)]=0.0000
  for index,row in topics_df.iterrows():
    topic_details=model.get_topic(row['Topic'])
    for (i,td) in enumerate(topic_details):
      topics_df.at[index,'w'+str(i)]=td[0]
      topics_df.at[index,'p'+str(i)]=td[1]
  topics_df['Author_Id']=Author_Id
  topics_df['author']=author
  clones_topics_df=clones_topics_df.append(topics_df) 

In [ ]:
clones_topics_df.columns

Index(['Topic', 'Count', 'Name', 'w0', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6',
       'w7', 'w8', 'w9', 'p0', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8',
       'p9', 'Author_Id', 'author'],
      dtype='object')

In [ ]:
clones_topics_df=clones_topics_df[['Author_Id', 'author','Topic', 'Count', 'Name', 'w0', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6','w7', 'w8', 'w9', 'p0', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8','p9']]

In [ ]:
clones_topics_df

,Author_Id,author,Topic,Count,Name,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9
0,69715,lizon **,-1,187,-1_написать_смотреть_свой_автор,написать,смотреть,свой,автор,муж,ваш,писать,думать,видеть,женщина,0.017130,0.015701,0.015288,0.014782,0.013083,0.012341,0.012287,0.009947,0.009658,0.009406
1,69715,lizon **,0,138,0_сын_ребёнок_муж_внучка,сын,ребёнок,муж,внучка,дочка,мальчик,мама,детей,младший,развод,0.054861,0.038308,0.029138,0.023043,0.019599,0.015614,0.015614,0.014015,0.012682,0.012609
2,69715,lizon **,1,78,1_писать_жизнь_форум_написать,писать,жизнь,форум,написать,энергию,сказать,стать,энергия,думать,личность,0.021210,0.017947,0.016679,0.015013,0.013648,0.013580,0.013498,0.013224,0.012915,0.012866
3,69715,lizon **,2,63,2_жить_день_париж_город,жить,день,париж,город,метро,дом,такси,шампанское,жизнь,работать,0.042163,0.029427,0.025196,0.025097,0.020241,0.019399,0.016086,0.015272,0.014885,0.014632
4,69715,lizon **,3,41,3_фотка_вечер_смотреть_танцевать,фотка,вечер,смотреть,танцевать,париж,рыск,рок,диско,светеллка,увидеть,0.030479,0.024431,0.023948,0.022767,0.021054,0.020318,0.018786,0.016262,0.015029,0.013254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,424711,Joconda Mona Liza _,0,64,0_жить_страна_дом_ездить,жить,страна,дом,ездить,сын,место,старший,деревня,день,поездка,0.029376,0.025318,0.024365,0.023660,0.021339,0.019286,0.017469,0.017155,0.016287,0.014516
1,424711,Joconda Mona Liza _,3,63,3_жизнь_ваш_автор_жить,жизнь,ваш,автор,жить,писать,жизни,свой,бог,читать,лично,0.055763,0.038526,0.030043,0.028125,0.025249,0.020552,0.020180,0.015868,0.014732,0.014170
2,424711,Joconda Mona Liza _,-1,43,-1_день_метро_женщина_комната,день,метро,женщина,комната,работа,работать,писать,делать,например,кровать,0.029625,0.028110,0.028110,0.026433,0.024212,0.023540,0.020909,0.020726,0.019616,0.018368
3,424711,Joconda Mona Liza _,2,35,2_молоко_супермаркет_покупка_одежда,молоко,супермаркет,покупка,одежда,евро,упаковка,покупать,купить,взнос,багажник,0.044592,0.038843,0.026684,0.025958,0.022876,0.022876,0.022035,0.019819,0.017970,0.017970


In [ ]:
SaveToExperimentLog(Experiments_file, 'all_lizon_clones', clones_topics_df)

## All authos together Topic Modelling


In [ ]:
df['Timestamp']=pd.to_datetime(df['Timestamp'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
data = df['preprocessed_message'].tolist()
timestamps = df['Timestamp'].tolist()

In [ ]:
topic_model = BERTopic(language="Russian",nr_topics=20)
#topic_model = BERTopic(embedding_model='blinoff/roberta-base-russian-v0',nr_topics=5)
topics, _ = topic_model.fit_transform(data)

Downloading:   0%|          | 0.00/968 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
freq = topic_model.get_topic_info()
freq

,Topic,Count,Name
0,-1,2600,-1_муж_сын_человек_ребёнок
1,0,406,0_писать_написать_аноним_читать
2,1,360,1_автор_написать_фотоальбомдата_01
3,2,216,2_школа_класс_учиться_урок
4,3,211,3_танцевать_танец_собака_диско
5,4,172,4_маска_лицо_фотка_красота
6,5,156,5_россия_российский_русский_путин
7,6,140,6_пляж_страна_море_индия
8,7,140,7_париж_парижа_метро_город
9,8,136,8_врач_рак_лекарство_хирург


In [ ]:
SaveToExperimentLog(Experiments_file, 'all_lizon_clones2', freq)

In [ ]:
fig = topic_model.visualize_topics()
fig

## Topics over Time

In [ ]:
topics_over_time = topic_model.topics_over_time(docs=data, 
                                                topics=topics, 
                                                timestamps=timestamps, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=5)

## Topics Entitlement

In [5]:
topic_keywords = [ 'муж',
 'сын',
 'внучка',
 'развод',
 'беременность',
 'дом',
 'сосед',
 'школа',
 'колледж',
 'пиано',
 'танец',
 'маска',
 'одежда',
 'фотка',
 'религия',
 'россия',
 'париж',
 'самолёт',
 'путешествие',
 'врач',
 'болезнь']

In [8]:
!pip install -q transformers

     |████████████████████████████████| 3.1 MB 30.7 MB/s 
     |████████████████████████████████| 596 kB 71.2 MB/s 
     |████████████████████████████████| 3.3 MB 61.0 MB/s 
     |████████████████████████████████| 895 kB 65.9 MB/s 
     |████████████████████████████████| 61 kB 557 kB/s 


In [51]:
from transformers import pipeline

ModuleNotFoundError: ignored

In [8]:
model = 'cointegrated/rubert-base-cased-nli-threeway'
classifier = pipeline('zero-shot-classification', model=model)

Downloading:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.49M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
long_messages=df[df['message_words']>=25][['Message_Id','message']].copy(deep=True)

In [10]:
long_messages['message'].fillna('missing')
posts = long_messages['message'].tolist()
posts=['missing' if x is np.nan else x for x in posts]

In [11]:
labels=classifier(posts, topic_keywords, multi_label=True)

In [12]:
topics=[]
for l in labels:
  topic_map={'Topic0':l['labels'][0], 'Score0':l['scores'][0],'Topic1':l['labels'][1],'Score1':l['scores'][1],'Topic2':l['labels'][2],'Score2':l['scores'][2]}
  topics.append(topic_map)

In [13]:
topics_df=pd.DataFrame.from_records(topics)
long_messages=long_messages.reset_index()
long_messages=long_messages['Message_Id']
long_messages=pd.concat([long_messages.reset_index(), topics_df], axis=1)
df = pd.merge(df, long_messages[['Message_Id', 'Topic0', 'Score0', 'Topic1','Score1', 'Topic2', 'Score2']], left_on='Message_Id', right_on='Message_Id', how='left')

In [6]:
df.head()

,Message_Id,Timestamp,Topic_1st_Message,Parent_Id,Author_Id,author,Topic_Id,Topic,Chapter_Id,Chapter,original_message,message,cnt_childs,cnt_immediate_childs,message_words,Topic0,Score0,Topic1,Score1,Topic2,Score2
0,56799126,2010-05-12 22:03:00,N,0,69715,lizon **,2281736,как часто вы покупаете ребенку игрушки?,137,Детская психология и развитие,"<div class=""body"">Каждую неделю, на барахолках...","Каждую неделю, на барахолках, где-то раз в мес...",1,1,21,NaN,NaN,NaN,NaN,NaN,NaN
1,57921402,2010-07-01 12:38:00,N,0,69715,lizon **,2322960,про негра,137,Детская психология и развитие,"<div class=""body"">Хе, а у нас, где мы жили ран...","Хе, а у нас, где мы жили раньше(свят свят, что...",0,0,71,фотка,0.728377,сын,0.620459,колледж,0.408833
2,58468210,2010-08-02 14:42:00,N,0,69715,lizon **,2350112,Как вы ходите с детьми в рестораны?,137,Детская психология и развитие,"<div class=""body"">Начали ходить с дитем в рест...",Начали ходить с дитем в рестораны лет с четыр...,0,0,16,NaN,NaN,NaN,NaN,NaN,NaN
3,58468241,2010-08-02 14:43:00,N,0,69715,lizon **,2351751,как спят ваши дети в жару,137,Детская психология и развитие,"<div class=""body"">Укладываю в жару спать в отк...",Укладываю в жару спать в открытой палатке в са...,0,0,14,NaN,NaN,NaN,NaN,NaN,NaN
4,58795095,2010-08-19 22:47:00,Y,0,69715,lizon **,2368698,читаь молитву на ночь в 5 лет-не слишком ли?,137,Детская психология и развитие,"<div class=""body"">У меня муж очень верующий ка...",У меня муж очень верующий католик. Я атеистка....,50,17,110,муж,0.817741,беременность,0.686566,религия,0.574169


In [15]:
df.to_csv(Messages_full_filename, header=True, index=False)

In [9]:
df['Score0']=df['Score0'].fillna(0.0)
df['Score1']=df['Score1'].fillna(0.0)
df['Score2']=df['Score2'].fillna(0.0)

In [14]:
df['Score0'].max()

0.9966254234313964

In [26]:
df[df['Score0']>0.95][['Message_Id','message','Topic0','Score0','Topic1','Score1','Topic2','Score2']].sort_values('Score0').tail(20)

,Message_Id,message,Topic0,Score0,Topic1,Score1,Topic2,Score2
3702,97730607,"Автор, трудно очень трудно самоубиться людям- ...",путешествие,0.997506,болезнь,0.963895,фотка,0.906789
2523,96050614,"Расскажу что сегодня было. В Париже , на Восто...",пиано,0.997596,фотка,0.997200,одежда,0.996625
3690,97742551,"Автор, вы боитесь что у вас онкология? Вот чуд...",болезнь,0.997600,путешествие,0.857438,беременность,0.573529
6217,102368591,"Охотно поясню. Он работает-следователем, в Пар...",фотка,0.997611,путешествие,0.918859,муж,0.880598
2489,95768861,Расскажу еще один случай. Это тоже случилось ...,путешествие,0.997637,фотка,0.980628,самолёт,0.962127
2670,97679822,Автор.... 1 класс? Девочка? Да еще он с вами т...,беременность,0.997656,развод,0.897966,муж,0.857084
3277,94052962,А вот в тему. У меня тут одна русская знакомая...,беременность,0.997657,фотка,0.984670,путешествие,0.744913
3030,92450580,Вот- вот.Мне непосчасливилось согласиться на в...,путешествие,0.997691,самолёт,0.992009,фотка,0.947402
1760,76333760,"Автор, если у вас в жизни есть люди. близкие ....",беременность,0.997805,болезнь,0.967788,внучка,0.837411
9140,81589116,Камелька вот схематично. Он учился в лицее. Пр...,беременность,0.997844,сын,0.740575,внучка,0.684080


In [5]:
topic_posts=' '.join(df[(((df['Score0']>0.95) & (df['Topic0']=='внучка')) | ((df['Score1']>0.9) & (df['Topic1']=='внучка')) | ((df['Score2']>0.88) & (df['Topic2']=='внучка')))]['message'])

In [6]:
len(df[(((df['Score0']>0.95) & (df['Topic0']=='внучка')) | ((df['Score1']>0.9) & (df['Topic1']=='внучка')) | ((df['Score2']>0.88) & (df['Topic2']=='внучка')))]['message'])

38

##Topics Summarization

In [9]:
%tensorflow_version 2.x

In [10]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 29.7 MB/s eta 0:00:01


In [11]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

In [12]:
model_name = 'IlyaGusev/mbart_ru_sum_gazeta'

In [14]:
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/3.23G [00:00<?, ?B/s]

In [ ]:
topics_summary=[]
for t in df[df['Score0']>0.95]['Topic0'].unique():
  topic_posts=' '.join(df[(((df['Score0']>0.99) & (df['Topic0']==t)) | ((df['Score1']>0.99) & (df['Topic1']==t)) | ((df['Score2']>0.99) & (df['Topic2']==t)) )]['message'])
  input_ids = tokenizer.prepare_seq2seq_batch(
    [topic_posts],
    src_lang="en_XX", # fairseq training artifact
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=1000
  )["input_ids"]

  output_ids = model.generate(
    input_ids=input_ids,
    max_length=1000,
    no_repeat_ngram_size=5,
    num_beams=5,
    top_k=0
  )[0]

  summary = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  topics_summary.append({'topic':t,'summary':summary})

In [37]:
topics_summary_df=pd.DataFrame(topics_summary)
topics_summary_df.head()

,topic,summary
0,путешествие,"Начну с того, что я не знаю, что вы говорите. ..."
1,одежда,В Париже установили пианоно. Для всех желающих...
2,фотка,В преддверии новогодних праздников я решила по...
3,беременность,"Начну с того, что я не люблю пить гормоны. Мне..."
4,внучка,"Начну с того, что я не знаю, как жить в Париже..."


In [38]:
SaveToExperimentLog(Experiments_file, 'Topics Summarys4', topics_summary_df)